In [4]:
pip install torchviz

  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4151 sha256=edcaab6cc69d2c9abbc0f741e2599f86475c1e50af4dcaea04378d1feea5c7ef
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install hiddenlayer

Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tarfile
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import time
from torchviz import make_dot
import hiddenlayer as hl
from collections import Counter

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
from torchvision import transforms
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler, WeightedRandomSampler

projekt="RESNET34"

In [7]:
pip install -q kaggle #Install kaggle

Note: you may need to restart the kernel to use updated packages.


In [8]:
!pip install opendatasets

In [9]:
import opendatasets as od

#volatille
#b06bccd3d1795d29ff0b071f635f2d79

od.download("https://www.kaggle.com/paultimothymooney/breast-histopathology-images", force=True) 

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  volatille


Your Kaggle Key:

  ································


100%|██████████| 3.10G/3.10G [00:18<00:00, 181MB/s] 


In [10]:
#Wczesniej wczytywałem poprzez sciezkie w systemie, po migracji na coolab zmieniłem ścieżkę.

data_dir = './breast-histopathology-images/'
folder_name = "IDC_regular_ps50_idx5"
image_folders = os.path.join(data_dir, folder_name)

transform = transforms.Compose([transforms.Resize((50, 50)), transforms.ToTensor()]) #Funkcja robie resize każdego pliku (50x50) oraz rzutuję na = torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0] 

zdjecia_idc = []
for file in os.listdir(image_folders):
    zdjecia_idc.append(ImageFolder(os.path.join(image_folders, file), transform=transform)) #pętla, alokacja wczesniej przyjętych zmian dla plików.
datasets = torch.utils.data.ConcatDataset(zdjecia_idc) #upchanie wszystkiego do datasets

In [11]:
from collections import Counter

#Obliczanie ile jest danych w zbiorze (IDC)
j=0
for dataset in tqdm(datasets.datasets):
    if j == 0:
        result = Counter(dataset.targets)
        j += 1
    else:
        result += Counter(dataset.targets)


Pliki_2 = os.listdir(os.path.join(data_dir, folder_name))
print(" \n Liczba Pacjentów:", len(Pliki_2))

print("""\n Liczba zdjęć:
    IDC_0 (Brak IDC)  : {}
    IDC_i (Obecne IDC): {}""".format(result[0], result[1]))


  0%|          | 0/279 [00:00<?, ?it/s]

 
 Liczba Pacjentów: 279

 Liczba zdjęć:
    IDC_0 (Brak IDC)  : 198738
    IDC_i (Obecne IDC): 78786


In [12]:
# Przygotowywanie modelu,danych itp.
random_seed = 42
torch.manual_seed(random_seed)

test_size = 38000
train_size = len(datasets) - test_size
train_ds, test_ds = random_split(datasets, [train_size, test_size])

val_size = 38000
train_size = len(train_ds) - val_size
train_ds, val_ds = random_split(train_ds, [train_size, val_size])

len(train_ds), len(val_ds), len(test_ds)

(201524, 38000, 38000)

In [14]:
#Dane do treningu, validacji oraz testu
#   shuffle=True, przy false może nastąpić większy błąd val_loss (w treningu sieci)
#   pin_memory to wstawka dla conda cpu

train_data = DataLoader(train_ds, shuffle=True, num_workers=2, pin_memory=True) 
val_data = DataLoader(val_ds, shuffle=True, num_workers=2, pin_memory=True)
test_data = DataLoader(test_ds, shuffle=True, num_workers=2, pin_memory=True)

In [15]:
# Transformacja danych /tt.ToTensor 
train_transform = tt.Compose([tt.ToTensor()])
valid_transform = tt.Compose([tt.ToTensor()])

In [16]:
# Batch Size
batch_size = 100

# PyTorch data loaders
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=3, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=3, pin_memory=True)
test_dl = DataLoader(test_ds, batch_size*2, num_workers=3, pin_memory=True)

In [17]:
### Funkcja wskaźnika F1 według algorytmu ###
#   Funkcja jest testem na dokładność. Wartość 1.0 oznacza najwyższą precyzję

#Funkcja oblicza wartości:
# TP = True Positives
# TN = True Negatives
# FP = False Positives
# FN = False Negatives

def F_score(output, label, threshold=0.5, beta=1):
    prob = output > threshold
    label = label > threshold

    TP = (prob & label).sum(1).float()
    TN = ((~prob) & (~label)).sum(1).float()
    FP = (prob & (~label)).sum(1).float()
    FN = ((~prob) & label).sum(1).float()

    precision = torch.mean(TP / (TP + FP + 1e-12))
    recall = torch.mean(TP / (TP + FN + 1e-12))
    F2 = (1 + beta**2) * precision * recall / (beta**2 * precision + recall + 1e-12)
    return F2.mean(0)

In [18]:
#Klasa zawierająca funkcję do kroku treningu modułu, kroku validacji modułu.
# @training_step() zwraca train_loss
# @validation_step() zwraca validation_loss
# @validation_epoch_end() 

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, targets = batch 
        targets = torch.reshape(targets.type(torch.cuda.FloatTensor), (len(targets), 1))
        out = self(images)                      
        loss = F.binary_cross_entropy(out, targets)      
        return loss
    
    def validation_step(self, batch):
        images, targets = batch
        targets = torch.reshape(targets.type(torch.cuda.FloatTensor), (len(targets), 1))
        out = self(images)                           # Generate predictions
        loss = F.binary_cross_entropy(out, targets)  # Calculate loss
        score = F_score(out, targets)
        return {'val_loss': loss.detach(), 'val_score': score.detach() }
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_scores = [x['val_score'] for x in outputs]
        epoch_score = torch.stack(batch_scores).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_score': epoch_score.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.4f}, train_loss: {:.4f}, val_loss: {:.4f}, val_score: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_score']))

In [19]:
#Serce Resnet34. 

class BreastCancerResnet34(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        # Use a pretrained model
        self.network = models.resnet34(pretrained=True)
        # Replace last layer
        num_ftrs = self.network.fc.in_features
        self.network.fc = nn.Linear(num_ftrs, 1)
    
    def forward(self, xb):
        return torch.sigmoid(self.network(xb))
    
    def freeze(self):
        # To freeze the residual layers
        for param in self.network.parameters():
            param.require_grad = False
        for param in self.network.fc.parameters():
            param.require_grad = True
    
    def unfreeze(self):
        # Unfreeze all layers
        for param in self.network.parameters():
            param.require_grad = True

In [20]:
def get_default_device():
    """CUDA GPU lub jesli nie dostepne to CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

device = get_default_device()
device

device(type='cuda')

In [21]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
test_dl = DeviceDataLoader(test_dl, device)

In [22]:
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in tqdm(train_loader):
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [23]:
#Wybieramy dane do modelu sieci, dodajemy urzadzenie(GPU/CPU)
#  RESNET32

model = to_device(BreastCancerResnet34(), device)

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

In [24]:
# Przypisujemy zwracaną walidację przez funkcję evaluate naszego modelu 
history = [evaluate(model, val_dl)]
history

[{'val_loss': 0.6400343179702759, 'val_score': 0.02352631650865078}]

In [25]:
#Blokujemy model zamrażanie zapobiega modyfikowaniu wag warstwy sieci neuronowej podczas wstecznego przebiegu uczenia. 

model.freeze()

In [26]:
#Liczba epok
epochs = 15

#Max learning rate
max_lr = 0.01

#Przycinanie gradientu, w celach optymalizacyjnych ai
grad_clip = 0.1

#Parametr dzięki któremu możemy osiągnąć mniejsze wagi modelu
weight_decay = 1e-4

#Funkcja optymalizacji ai - Adam
opt_func = torch.optim.Adam

In [27]:
%%time
start_time = time.time()
history += fit_one_cycle(epochs, max_lr, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)

train_time = time.time() - start_time

  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [0], last_lr: 0.0015, train_loss: 0.3061, val_loss: 0.7164, val_score: 0.0917


  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [1], last_lr: 0.0044, train_loss: 0.3091, val_loss: 0.3615, val_score: 0.2383


  0%|          | 0/2016 [00:00<?, ?it/s]

...

Epoch [12], last_lr: 0.0009, train_loss: 0.2889, val_loss: 0.2982, val_score: 0.2329


  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [13], last_lr: 0.0002, train_loss: 0.2853, val_loss: 0.2883, val_score: 0.2134


  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [14], last_lr: 0.0000, train_loss: 0.2826, val_loss: 0.2878, val_score: 0.2139
CPU times: user 25min 37s, sys: 1min 18s, total: 26min 56s
Wall time: 39min 53s


In [28]:
model.unfreeze()

In [29]:
%%time
start_time = time.time()
history += fit_one_cycle(epochs, 0.001, model, train_dl, val_dl, 
                         grad_clip=grad_clip, 
                         weight_decay=weight_decay, 
                         opt_func=opt_func)
train_time += time.time() - start_time

  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [0], last_lr: 0.0002, train_loss: 0.2824, val_loss: 0.2890, val_score: 0.2244


  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [1], last_lr: 0.0004, train_loss: 0.2830, val_loss: 0.2917, val_score: 0.2045


  0%|          | 0/2016 [00:00<?, ?it/s]

...

Epoch [12], last_lr: 0.0001, train_loss: 0.2740, val_loss: 0.2842, val_score: 0.2231


  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [13], last_lr: 0.0000, train_loss: 0.2728, val_loss: 0.2833, val_score: 0.2162


  0%|          | 0/2016 [00:00<?, ?it/s]

Epoch [14], last_lr: 0.0000, train_loss: 0.2723, val_loss: 0.2830, val_score: 0.2167
CPU times: user 25min 17s, sys: 1min 16s, total: 26min 33s
Wall time: 39min 9s


In [35]:
#
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)    
    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [36]:
# 
IDC_PRED=[]
IDC_TRUE=[]
with torch.no_grad():
    with tqdm(test_dl, unit="batch") as tepoch:
        for inp, labels in tepoch:
            inp, labels = inp.to(device), labels.to(device)
            y_test_pred = model(inp)
            acc = multi_acc(y_test_pred, labels)
            _, y_pred_tag = torch.max(y_test_pred, dim = 1)
            tepoch.set_postfix(accuracy = acc.item())
            IDC_PRED.append(y_pred_tag.cpu().numpy())
            IDC_TRUE.append(labels.cpu().numpy())

  0%|          | 0/190 [00:00<?, ?batch/s]

In [37]:
# flatten prediction and true lists
flat_pred = []
flat_true = []
for i in range(len(IDC_PRED)):
    for j in range(len(IDC_PRED[i])):
        flat_pred.append(IDC_PRED[i][j])
        flat_true.append(IDC_TRUE[i][j])
        
print(f"number of testing samples results: {len(flat_pred)}")

number of testing samples results: 38000


In [38]:
#Testing accuracy. Wynik dokładności ai

print(f"Dokładność wynosi: {accuracy_score(flat_true, flat_pred) * 100:.2f}%")

Dokładność wynosi: 71.48%


In [ ]:
# Macierz błędu

print(classification_report(flat_true, flat_pred))